Read in JSON files containing the dialogs

In [117]:
import json
from pandas.io.json import json_normalize
import os
import logging
import logging.handlers
import pandas as pd
import numpy as np
import glob
from datetime import datetime,timedelta
from collections import Counter
import nltk 


In [187]:
def dt_parse(t):
    ret = datetime.strptime(t[0:16],'%Y-%m-%dT%H:%M')
    return ret 

# Nasoto DataFrame

In [188]:
rootDirNasoto = 'webchat-nasoto-pilot/' 

jsdata_nasoto = []

df_nasoto = pd.DataFrame(pd.np.empty((0, 7)))
df_nasoto.columns = ['Username','Duration', 'Chatbot', 'NumberOfTurns', 'Utterance','Number of words', 'Number of chars']

for subdir, dirs, files in os.walk(rootDirNasoto): 
    for dire in dirs:
        arrTime = []
        arrUtterance = []
        numberOfTurns = 0
        count = 0
        count_ch = 0
        for file in glob.glob(rootDirNasoto+dire+'/'+'*.json'):
            with open(file) as f:
                countChar = 0
                data = json.load(f)
                numberOfTurns += 1
                arrTime.append( dt_parse(data['timestamp']))
                arrUtterance.append(data['text'])
                for word in (data['text']).split():
                    count += 1
                    for char in word:
                        count_ch += (len(char))
                    countChar = count_ch
            
        
        
        #add post interaction survey (Nasoto)
        
        #Duration
        maxTime = max(dt for dt in arrTime )
        minTime = min(dt for dt in arrTime)
        duration =  maxTime - minTime
        
        #get the information from makoto inserted
        jsdata_nasoto.append({'Username': data['from']['name'],
                       'Duration': duration,'Chatbot': data['recipient']['name'], 
                       'NumberOfTurns' : numberOfTurns, 'Utterance': arrUtterance, 'Number of words': count,
                             'Number of chars': countChar})



df_nasoto =  pd.DataFrame.from_records(jsdata_nasoto)
df_nasoto[:]['Duration'] / np.timedelta64(1, 'm')  
df_nasoto

/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



,Username,Duration,Chatbot,NumberOfTurns,Utterance,Number of words,Number of chars
0,sorcha.nicconmara,00:08:00,Nasoto-Bot,19,"[I now study from home, I think it was the rig...",68,303
1,charles.kelly1,00:07:00,Nasoto-Bot,42,"[Playing basketball, No thank you, No thank yo...",101,469
2,chloe.foxe,00:04:00,Nasoto-Bot,23,"[I am taking 5 modules this semester, Nah, ext...",84,394
3,elizabeth.burke,00:03:00,Nasoto-Bot,17,"[12 modules, no, no, 12, It's fun, my favourit...",48,234


# Makoto DataFrame

In [189]:
rootDirMakoto = 'webchat-makoto-pilot/'
jsdata_makoto = []

df_makoto = pd.DataFrame(pd.np.empty((0, 7)))
df_makoto.columns = ['Username','Duration', 'Chatbot', 'NumberOfTurns', 'Utterance','Number of words', 'Number of chars']

for subdir, dirs, files in os.walk(rootDirMakoto): 
    for dire in dirs:
        arrTime = []
        arrUtterance = []
        numberOfTurns = 0
        count = 0
        count_ch = 0
        for file in glob.glob(rootDirMakoto+dire+'/'+'*.json'):
            with open(file) as f:
                countChar = 0
                data = json.load(f)
                numberOfTurns += 1
                arrTime.append( dt_parse(data['timestamp']))
                arrUtterance.append(data['text'])
                for word in (data['text']).split():
                    count += 1
                     for char in word:
                        count_ch += (len(char))
                    countChar = count_ch
                    
        #information for the google form
        #add post interaction survey (Makoto)
        #Duration
        maxTime = max(dt for dt in arrTime )
        minTime = min(dt for dt in arrTime)
        duration =  maxTime - minTime
        
        #get the information from makoto inserted
        jsdata_makoto.append({'Username': data['from']['name'],
                       'Duration': duration,'Chatbot': data['recipient']['name'], 
                       'NumberOfTurns' : numberOfTurns, 'Utterance': arrUtterance, 'Number of words': count,
                              'Number of chars': countChar})
df_makoto =  pd.DataFrame.from_records(jsdata_makoto) 
df_makoto[:]['Duration'] / np.timedelta64(1, 'm')

df_makoto

IndentationError: unexpected indent (<ipython-input-189-9ee47ddc3b79>, line 23)

DataFrame for Pre-Interacrion Survey

In [190]:
PreInteractionDir = 'surveys-pilot/Pre-InteractionSurvey-Pilot.csv'
df_Survey1 = pd.read_csv(PreInteractionDir,  error_bad_lines=False)
df_Survey1['Username'] =  df_Survey1['Username'].str.replace(r'@ucdconnect.ie', '')

Clean Data

In [191]:
df_Survey1 = df_Survey1.drop([ "Total score", "Age [Score]", "Age [Feedback]", 
                              "Gender [Score]" , "Gender [Feedback]", "Do you know what a chatbot is? [Score]", 
                              "Do you know what a chatbot is? [Feedback]", 
                              "If Yes, describe your understanding of what a chatbot is  [Score]",
                            "If Yes, describe your understanding of what a chatbot is  [Feedback]", "How often do you use Siri, Alexa, Google Assistant or any other virtual assistant? [Score]",
                            "How often do you use Siri, Alexa, Google Assistant or any other virtual assistant? [Feedback]",
                             "I am reserved [Score]", "I am reserved [Feedback]", "I tend to find fault with others [Score]",
                            "I tend to find fault with others [Feedback]", "I am helpful and unselfish with others [Score]",
                            "I am helpful and unselfish with others [Feedback]", "I am full of energy [Score]",
                            "I am full of energy [Feedback]", "I start quarrels with others [Score]",
                            "I start quarrels with others [Feedback]", "I generate a lot of enthusiasm  [Score]",
                            "I generate a lot of enthusiasm  [Feedback]","I have a forgiving nature [Score]",
                            "I have a forgiving nature [Feedback]", "I tend to be quiet  [Score]",
                            "I tend to be quiet  [Feedback]", "I am generally trusting [Score]",
                            "I am generally trusting [Feedback]", "I have an assertive personality [Score]",
                            "I have an assertive personality [Feedback]", "I can be cold and aloof  [Score]",
                            "I can be cold and aloof  [Feedback]", "I am sometimes shy, inhibited [Score]",
                            "I am sometimes shy, inhibited [Feedback]", "I am considerate and kind to almost everyone [Score]",
                            "I am considerate and kind to almost everyone [Feedback]", "I am outgoing, sociable  [Score]",
                            "I am outgoing, sociable  [Feedback]", "I am sometimes rude to others [Score]",
                            "I am sometimes rude to others [Feedback]", "I like to cooperate with others [Score]",
                            "I like to cooperate with others [Feedback]"], axis=1)

DataFrame for Post-Interaction Survey (Makoto)

In [192]:
PostMakotoDir = 'surveys-pilot/Post-InteractionSurveyMakoto-Pilot.csv'
df_PostMakoto = pd.read_csv(PostMakotoDir, error_bad_lines=False)
df_PostMakoto['Username'] =  df_PostMakoto['Username'].str.replace(r'@ucdconnect.ie', '')

DataFrame for Post-Interaction Survey (Nasoto)

In [193]:
SurveyNasotoDir = 'surveys-pilot/Post-InteractionSurveyNasoto-Pilot.csv'
df_surveyNasoto = pd.read_csv(SurveyNasotoDir)
df_surveyNasoto['Username'] =  df_surveyNasoto['Username'].str.replace(r'@ucdconnect.ie', '')


 DataFrame for Post-Interaction Survey/Final Survey (Both) 

In [194]:
Survey4Dir = 'surveys-pilot/FinalSurvey-Pilot.csv'
df_survey4 = pd.read_csv(Survey4Dir)
df_survey4['Username'] =  df_survey4['Username'].str.replace(r'@ucdconnect.ie', '')


Automation of personality assignment

In [195]:
extraversion = ((6 - df_Survey1['I am reserved']) 
        + df_Survey1['I am full of energy'] 
        + df_Survey1['I generate a lot of enthusiasm '] 
        + (6 - df_Survey1['I tend to be quiet ']) 
        + df_Survey1['I have an assertive personality'] 
        + (6 - df_Survey1['I am sometimes shy, inhibited']) 
        + df_Survey1['I am outgoing, sociable '])


df_Survey1["extraversion score"] = extraversion
               
               
agreeableness = ((6 - df_Survey1['I tend to find fault with others']) 
        + df_Survey1['I am helpful and unselfish with others']
        + (6 - df_Survey1['I start quarrels with others'] )
        + df_Survey1['I have a forgiving nature'] 
        + df_Survey1['I am generally trusting'] 
        + (6 - df_Survey1['I can be cold and aloof '])
        + df_Survey1['I am considerate and kind to almost everyone']
        + ( 6 - df_Survey1['I am sometimes rude to others']) 
        + df_Survey1['I like to cooperate with others'])
df_Survey1["agreeableness score"] = agreeableness




In [196]:
### Merge all the surveys based on username (email) + merge with interaction data (remove @ucdconnect.ie from email using regex)
from functools import reduce
import re

In [223]:
df_allSurveys =  [df_Survey1, df_PostMakoto, df_surveyNasoto, df_survey4, df_nasoto, df_makoto]


df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Username'],
                                            how='outer'), df_allSurveys)
df_merged

,Timestamp_x,Username,Age,Gender,Do you know what a chatbot is?,"If Yes, describe your understanding of what a chatbot is","How often do you use Siri, Alexa, Google Assistant or any other virtual assistant?",I am reserved,I tend to find fault with others,I am helpful and unselfish with others,...,NumberOfTurns_x,Utterance_x,Number of words_x,Number of chars_x,Duration_y,Chatbot_y,NumberOfTurns_y,Utterance_y,Number of words_y,Number of chars_y
0,2020/04/01 10:45:16 am CET,sorcha.nicconmara,18-24,Female,Yes,A program/piece of software that imitates huma...,Never,2,3,4,...,19,"[I now study from home, I think it was the rig...",68,303,00:05:00,Makoto-Bot,15,"[extracurricular, The topic is very interestin...",39,215
1,2020/04/01 1:45:07 pm CET,charles.kelly1,18-24,Male,Yes,An ai that attempts to replicate human convers...,Somewhat,4,5,4,...,42,"[Playing basketball, No thank you, No thank yo...",101,469,00:02:00,Makoto-Bot,11,"[Yes ther is, Yes, I like animals, 4 or 5, Gam...",26,108
2,2020/04/01 3:11:18 pm CET,chloe.foxe,18-24,Female,Yes,A chatbot is a computer program with human cap...,Somewhat,4,2,4,...,23,"[I am taking 5 modules this semester, Nah, ext...",84,394,00:05:00,Makoto-Bot,10,"[Uhm. I like solving problems, yeah we have a ...",46,193
3,2020/04/01 6:44:12 pm CET,elizabeth.burke,18-24,Female,Yes,Artificial Intelligence capable of holding a c...,Somewhat,2,3,5,...,17,"[12 modules, no, no, 12, It's fun, my favourit...",48,234,00:09:00,Makoto-Bot,29,"[It's an inconvenience but necessary, health i...",88,416


In [198]:
#merge nasoto and makoto data 
frames = [df_nasoto, df_makoto]
df_chatbotdata =  pd.concat(frames)
df_chatbotdata

,Username,Duration,Chatbot,NumberOfTurns,Utterance,Number of words,Number of chars
0,sorcha.nicconmara,00:08:00,Nasoto-Bot,19,"[I now study from home, I think it was the rig...",68,303
1,charles.kelly1,00:07:00,Nasoto-Bot,42,"[Playing basketball, No thank you, No thank yo...",101,469
2,chloe.foxe,00:04:00,Nasoto-Bot,23,"[I am taking 5 modules this semester, Nah, ext...",84,394
3,elizabeth.burke,00:03:00,Nasoto-Bot,17,"[12 modules, no, no, 12, It's fun, my favourit...",48,234
0,chloe.foxe,00:05:00,Makoto-Bot,10,"[Uhm. I like solving problems, yeah we have a ...",46,193
1,elizabeth.burke,00:09:00,Makoto-Bot,29,"[It's an inconvenience but necessary, health i...",88,416
2,charles.kelly1,00:02:00,Makoto-Bot,11,"[Yes ther is, Yes, I like animals, 4 or 5, Gam...",26,108
3,sorcha.nicconmara,00:05:00,Makoto-Bot,15,"[extracurricular, The topic is very interestin...",39,215


# Summary Stats

In [199]:
import plotly.express as px

In [200]:
#merge df_nasoto with df_makoto
df_cd = df_nasoto.append(df_makoto)
# convert duratin to minutes from ns
df_cd = df_cd.astype({'Duration': 'timedelta64[m]'})

## Box Plot  Interaction Duration

In [201]:
fig = px.box(df_cd , "Chatbot", "Duration", "Chatbot")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

## BoxPlot Number of Turns

In [202]:
fig = px.box(df_cd , "Chatbot", "NumberOfTurns", "Chatbot")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

## BoxPlot Length of Utterances

In [203]:
fig = px.box(df_cd , "Chatbot", "Number of words", "Chatbot")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

## BoxPlot Rating of Personality 

In [204]:
fig = px.box(df_merged , "Chatbot_x", "How would you rate your experience/interaction with Nasoto based on the it's attitude/personality?", "Chatbot_x")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [205]:
fig = px.box(df_merged , "Chatbot_y", "How would you rate your experience with Makoto based on the its attitude/personality?", "Chatbot_y")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

### Demographic Visualisations 

In [206]:
# Age
fig = px.histogram(df_merged, x="Age")
fig.show()

In [207]:
# Gender
fig = px.histogram(df_merged, x='Gender')
fig.show()

In [208]:
# How often the use a chatbot
fig = px.histogram(df_merged, x='How often do you use Siri, Alexa, Google Assistant or any other virtual assistant?')
fig.show()

# RQ1: Does the personality demonstrated by a chatbot affect the user’s experience?


- Rating of experience based on the attitude/personality of the chatbot
- Anlayse perferred chatbot
- Explain why you preferred interacting with this chatbot? 
- Compare Duration (time & # of turns) for each chatbot
- Lenghth of utterances
- Look at whether their personality matches that of the preffered chatbot 




### Check if there's a correlation between the preffered chatbot and personality scores

In [209]:
df_personality = pd.DataFrame(pd.np.empty((0, 5)))
df_personality.columns = ['Extraversion Score', 'Agreeableness Score', 'Average score','Preffered_Chatbot', 'Chatbot_binary']

### using df_merged
df_personality['Extraversion Score'] = df_merged['extraversion score']
df_personality['Agreeableness Score']= df_merged['agreeableness score']
df_personality['Average score'] = (df_merged['extraversion score'] + df_merged['agreeableness score'])/2
df_personality['Preffered_Chatbot'] = df_merged['Which chatbot did you prefer interacting with?']


/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [210]:
df_personality['Chatbot_binary'] = pd.np.where(
df_personality.Preffered_Chatbot.str.contains("Makoto"), 0, 1)

df_personality

/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



,Extraversion Score,Agreeableness Score,Average score,Preffered_Chatbot,Chatbot_binary
0,25,36,30.5,Makoto,0
1,16,27,21.5,Makoto,0
2,23,31,27.0,Nasoto,1
3,30,38,34.0,Makoto,0


In [211]:
#Point Biserial Correlation 
from scipy.stats import pointbiserialr

In [212]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')

In [213]:
#Convert chatbot names to values 
#0 - Makoto 
#1 - Nasoto
chatbot = df_personality['Chatbot_binary']
extraversion_score = df_personality['Extraversion Score']
agreeableness_score = df_personality['Agreeableness Score']

#pbc of extraversion scores
pbc_extraversion = pointbiserialr(chatbot, extraversion_score)
pbc_agreeableness = pointbiserialr(chatbot, agreeableness_score)

print(pbc_extraversion)
print(pbc_agreeableness)


PointbiserialrResult(correlation=-0.0574484989621426, pvalue=0.9425515010378573)
PointbiserialrResult(correlation=-0.2684624220856098, pvalue=0.7315375779143902)


#### Is the total word count an indicator of their preffered chatbot?

#### Get total word count for each chatbot 

In [214]:
total_wc_makoto = df_merged['Number of words_x'].sum(axis = 0, skipna = True) 
total_wc_nasoto = df_merged['Number of words_y'].sum(axis = 0, skipna = True)

#### Average word Count 

In [215]:
avgWCMakoto = total_wc_makoto/ len(df_merged.index)
avgWCNasoto = total_wc_nasoto / len(df_merged.index)

#### Total Char Count 

In [226]:
total_cc_makoto = df_merged['Number of chars_x'].sum(axis = 0, skipna = True) 
total_cc_nasoto = df_merged['Number of chars_y'].sum(axis = 0, skipna = True)
total_cc_makoto

1400

#### Average char count 

In [232]:
avgCCMakoto = total_cc_makoto / len(df_merged.index)
avgCCNasoto = total_cc_nasoto / len(df_merged.index)

In [233]:
df_length =  pd.DataFrame(pd.np.empty((0, 5)))
df_length.columns = ['Utterance length Makoto', 'Number of Turns Makoto','Utterance length Nasoto','Number of Turns Nasoto', 'Preffered Chatbot']

df_length['Preffered Chatbot'] =  df_merged['Which chatbot did you prefer interacting with?']
df_length['Number of Turns Makoto'] = df_merged['NumberOfTurns_x']
df_length['Number of Turns Nasoto'] = df_merged['NumberOfTurns_y']
df_length['Utterance length Makoto'] = df_merged['Number of words_x']
df_length['Utterance length Nasoto'] = df_merged['Number of words_y']


/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [234]:
# data frame for utterance data 
utterance_data =[['Makoto',total_wc_makoto, avgWCMakoto, total_cc_makoto, avgCharCountMak],
             ['Nasoto',total_wc_nasoto, avgWCNasoto, total_cc_nasoto, avgCharCountNas]   ]

df_utterance = pd.DataFrame(utterance_data, columns = ['Chatbot', 'TotalWordCount', 'AverageNumberWords', 'TotalCharCount', 'AverageCharCount' ] )
df_utterance

,Chatbot,TotalWordCount,AverageNumberWords,TotalCharCount,AverageCharCount
0,Makoto,301,75.25,1400,350.0
1,Nasoto,199,49.75,932,233.0


#### Plot Word Count 

In [235]:
fig = px.line(df_chatbotdata, x=df_chatbotdata.index, y="Number of words",
              color='Chatbot') 
#average word count makoto 
fig.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'y', y0= avgWCMakoto, y1= avgWCMakoto,
      xref= 'paper', x0= 0, x1= 1,
      line_color="Blue"),
        dict( type= 'line',
      yref= 'y', y0= avgWCNasoto, y1= avgWCNasoto,
      xref= 'paper', x0= 0, x1= 1,
      line_color="Red"  
        
            )
    
])

fig.show()

#### Plot Char count 

In [236]:
fig = px.line(df_chatbotdata, x=df_chatbotdata.index, y="Number of chars",
              color='Chatbot') 
#average word count makoto 
fig.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'y', y0= avgCCMakoto, y1= avgCCMakoto,
      xref= 'paper', x0= 0, x1= 1,
      line_color="Blue"),
        dict( type= 'line',
      yref= 'y', y0= avgCCNasoto, y1= avgCCNasoto,
      xref= 'paper', x0= 0, x1= 1,
      line_color="Red"  
        
            )
    
])

fig.show()

In [237]:
# Linear Regression 
import statsmodels.api as sm


# RQ2: Can Personality be simulated by a chatbot?

- Analyse any descriptions of the chatbots for key words associated with personality
- look at response for: How would you rate your experience with Makoto based on the its attitude/personality? + expansion



In [83]:
import nltk 

In [86]:
#Using natural languge processing find the adjectives asscoiated with each chatbot
def find_adjective(keyword):
    if (pd.isnull(keyword)):
        return ''
    else:
        tokens = nltk.word_tokenize(keyword)
        tagged = nltk.pos_tag(tokens)
        adjective = [w for w,t in tagged if "JJ" in t]
        if len(adjective) < 1:
            return ''
        else:
            return adjective[0]



#### Adjectives associated with Makoto

In [104]:
makoto_adj =  []
makoto_adj = df_merged['Describe Makoto'].apply(find_adjective)
makoto_adj = makoto_adj.append(df_merged['Expand on your answer (optional).2_x'].apply(find_adjective))
#For preferred chatbot = Makoto
makoto_adj

0    enthusiastic
1                
2     inquisitive
3     interesting
0                
1        Pleasant
2                
3           clear
dtype: object

#### Adjectives associted with Nasoto

In [103]:
nasoto_adj =  []
nasoto_adj = (df_merged['Describe Nasoto'].apply(find_adjective))
nasoto_adj = nasoto_adj.append(df_merged['Expand on your answer (optional).2_y'].apply(find_adjective))

#For preferred chatbot = Nasoto
   
nasoto_adj

0    Positive
1     Robotic
2         own
3      direct
0            
1     neutral
2     overall
3      formal
dtype: object

# RQ3: Can a user’s personality be inferred through their interaction with a chatbot?

- NLP on the language used in all utterances (for both Chatbots)
- Compare personality assigned through test with these results